In [18]:
import numpy as np
import pandas as pd
import os

In [2]:
DATA_DIR = "./data"

In [3]:
df_train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"), index_col="id")
df_test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"), index_col="id")

In [4]:
df_train.head()

,reviews,target
id,,
0,조아요 처음구입 싸게햇어요,2
1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,튼튼하고 손목을 잘 받쳐주네요~,5


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  25000 non-null  object
 1   target   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 585.9+ KB


In [6]:
print("------train data------")
print(f"target 1 : {(df_train['target'] == 1).sum()}개, {round((df_train['target'] == 1).sum() / 25000 * 100)}%")
print(f"target 2 : {(df_train['target'] == 2).sum()}개, {round((df_train['target'] == 2).sum() / 25000 * 100)}%")
print(f"target 4 : {(df_train['target'] == 4).sum()}개, {round((df_train['target'] == 4).sum() / 25000 * 100)}%")
print(f"target 5 : {(df_train['target'] == 5).sum()}개, {round((df_train['target'] == 5).sum() / 25000 * 100)}%")

------train data------
target 1 : 4500개, 18%
target 2 : 8000개, 32%
target 4 : 2500개, 10%
target 5 : 10000개, 40%


In [7]:
review_length = df_train["reviews"].map(lambda x: len(x))

print(f'리뷰 길이 최대값 : {review_length.max()}')
print(f'리뷰 길이 최소값 : {review_length.min()}')
print(f'리뷰 길이 평균 : {review_length.mean()}')
print(f'리뷰 길이 표준편차 : {review_length.std()}')
print(f'리뷰 길이 중앙값 : {review_length.median()}')

리뷰 길이 최대값 : 140
리뷰 길이 최소값 : 5
리뷰 길이 평균 : 39.98808
리뷰 길이 표준편차 : 29.423689912292634
리뷰 길이 중앙값 : 29.0


## 접근 방식
- target이 1,2,4,5이기 때문에 multiclass 문제로 봐야 할 것 같음.
- CountVectorizer : 
  - `from sklearn.feature_extraction.text import CountVectorizer`
- konlpy
  - `from konlpy import Okt`
- TF-IDF
  - `from sklearn.feature_extraction.text import TfidfVectorizer`

### koNLPy 사용
1. `Okt`로 문장을 토큰화
2. `Okt.pos()`로 텍스트를 형태소 단위로 나눈 뒤 품사 tagging
3. 명사, 동사, 형용사, 부사, 숫자, KoreanParticle로 tagging
4. train/validation
4. `CountVectorizer`로 텍스트 벡터화
5. LogisticRegression 학습 및 validation set으로 모델 검증

### TF-IDF


In [31]:
okt = Okt()
tokens = okt.morphs((df_train.reviews[1]))
tokenize = " ".join(tokens)
tokenize

'생각 보다 잘 안 돼요 매지 바른지 하루 밖에 안 됐는데 ㅠㅠ 25천원 가량 주고 사기 너무 아깝네요'

In [14]:
pos = okt.pos(tokenize)
pos

[('생각', 'Noun'),
 ('보다', 'Verb'),
 ('잘', 'Verb'),
 ('안', 'Noun'),
 ('돼요', 'Verb'),
 ('매지', 'Verb'),
 ('바른지', 'Verb'),
 ('하루', 'Noun'),
 ('밖에', 'Josa'),
 ('안', 'Noun'),
 ('됐는데', 'Verb'),
 ('ㅠㅠ', 'KoreanParticle'),
 ('25천원', 'Number'),
 ('가량', 'Noun'),
 ('주고', 'Verb'),
 ('사기', 'Noun'),
 ('너무', 'Adverb'),
 ('아깝네요', 'Adjective')]

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit([tokenize])

CountVectorizer()

In [37]:
print(vectorizer.vocabulary_)

{'생각': 11, '보다': 9, '돼요': 4, '매지': 6, '바른지': 7, '하루': 14, '밖에': 8, '됐는데': 5, 'ㅠㅠ': 1, '25천원': 0, '가량': 2, '주고': 13, '사기': 10, '너무': 3, '아깝네요': 12}
